* 相比于其他机器学习模型，决策树在模型描述上拥有巨大优势。决策树的推断逻辑非常直观，具有清晰的可解释性，也方便的了模型的可视化。这些特性同时也保证在使用决策树模型是，时无需考虑对数据的量化，甚至是标准化的。 并且和之前K-Nearest Neighbor Classifier不同，决策树仍属于参数模型，需要花费更多的时间在训练数据上。

In [1]:
import pandas as pd

In [2]:
titanic = pd.read_csv('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.txt')

In [3]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
row.names    1313 non-null int64
pclass       1313 non-null object
survived     1313 non-null int64
name         1313 non-null object
age          633 non-null float64
embarked     821 non-null object
home.dest    754 non-null object
room         77 non-null object
ticket       69 non-null object
boat         347 non-null object
sex          1313 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 112.9+ KB


In [4]:
X=titanic[['pclass','age','sex']]
y=titanic['survived']

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       633 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


* 👆age这个数据列，只有633个，需要补完
* sex和pclass两个数据列的之都是类别性的，需要转化为数值特征，用0/1表示
* 我们补充age里的数据，使用平均数或者中位数都是对模型偏离造成最小影响的策略

In [6]:
# y

In [7]:
X['age'].fillna(X['age'].mean(),inplace=True) # 原数据已被填充

C:\Users\wqxsh\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       1313 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=33)

In [11]:
#X_train

In [12]:
from sklearn.feature_extraction import DictVectorizer

In [13]:
vec = DictVectorizer(sparse=False)

In [14]:
X_train = vec.fit_transform(X=X_train.to_dict(orient='record')) # convert the DataFrame to a dictionary orient='record':list like [{column->value}...{clolun->value}]

In [15]:
vec.feature_names_

['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', 'sex=female', 'sex=male']

In [16]:
X_test = vec.transform(X=X_test.to_dict(orient='record'))

In [17]:
from sklearn.tree import DecisionTreeClassifier

In [18]:
dct = DecisionTreeClassifier()

In [19]:
dct.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [20]:
dct_y_predict = dct.predict(X=X_test)

In [21]:
print("The accuracy of DecisionTreeClassifier is : "+str(dct.score(X=X_test,y=y_test)))

The accuracy of DecisionTreeClassifier is : 0.7811550151975684


In [27]:
from sklearn.metrics import classification_report

In [28]:
print(classification_report(y_true=y_test,y_pred=dct_y_predict,target_names=['died','survived']))

              precision    recall  f1-score   support

        died       0.78      0.91      0.84       202
    survived       0.80      0.58      0.67       127

   micro avg       0.78      0.78      0.78       329
   macro avg       0.79      0.74      0.75       329
weighted avg       0.78      0.78      0.77       329



In [29]:
from sklearn.tree import export_graphviz
import graphviz

In [30]:
tree_pic = export_graphviz(dct,out_file="mytree.pdf")
with open('mytree.pdf') as f:
    dot_graph = f.read()


In [31]:
graphviz.Digraph(dot_graph) #不知道这问题出在哪了

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH